In [ ]:
import numpy as np
from ice_interp import ice_interp_mod


# Define input values
lon = np.float64(255.070312500000)  # Interpolation point longitude (degrees)
lat = np.float64(74.4735717773438)  # Interpolation point latitude (degrees)

x_corners = np.array([253.978338312172, 254.698325218660, 256.152459376863, 255.464594303337], dtype=np.float64)
y_corners = np.array([74.4775053981321, 74.2803767364551, 74.6534788353970, 74.8496146897753], dtype=np.float64)
p = np.array([0.04222093679422526, 0.02939444349332336, 0.02361927128232367, 0.02457275269829678], dtype=np.float64)  # Values at the quadrilateral corners

# Prepare output variable
expected_obs = np.zeros(1, dtype=np.float64)

# Initialize the Fortran module
ice_interp_mod.init()

# this call is so init location_mod is called, and does all its print outs
expected_obs = ice_interp_mod.quad_idw_interp_f2py(
        lon, lat, x_corners, y_corners, p
)




 --------------------------------------
 Starting ... at YYYY MM DD HH MM SS = 
                 2025  4  1 15 28 12
 --------------------------------------

  set_nml_output No echo of NML values
  location_mod: using code with optimized cutoffs
  location_mod: Including vertical separation when computing distances:
  location_mod:        # pascals ~ 1 horiz radian:      100000.00000
  location_mod:         # meters ~ 1 horiz radian:       10000.00000
  location_mod:   # model levels ~ 1 horiz radian:          20.00000
  location_mod:  # scale heights ~ 1 horiz radian:           5.00000


In [2]:
# Call the Fortran subroutine
for i in range(10):
    expected_obs = ice_interp_mod.quad_idw_interp_f2py(
        lon, lat, x_corners, y_corners, p
    )
    # Print the result
    print(f"Interpolated value at ({lon}, {lat}): {expected_obs}")


Interpolated value at (255.0703125, 74.4735717773438): 0.03081763984660563
Interpolated value at (255.0703125, 74.4735717773438): 0.03081763984660563
Interpolated value at (255.0703125, 74.4735717773438): 0.03081763984660563
Interpolated value at (255.0703125, 74.4735717773438): 0.03081763984660563
Interpolated value at (255.0703125, 74.4735717773438): 0.03081763984660563
Interpolated value at (255.0703125, 74.4735717773438): 0.03081763984660563
Interpolated value at (255.0703125, 74.4735717773438): 0.03081763984660563
Interpolated value at (255.0703125, 74.4735717773438): 0.03081763984660563
Interpolated value at (255.0703125, 74.4735717773438): 0.03081763984660563
Interpolated value at (255.0703125, 74.4735717773438): 0.03081763984660563
